In [2]:
try:
    ray.shutdown()
except:
    pass

%load_ext autoreload
%autoreload 2

import gym, ray
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms import sac
from ray.rllib.algorithms import pg
from ray.rllib.algorithms import a3c
from ray.rllib.algorithms import td3

ray.init()


import os
import fmuSimulation.gymFMU as ExampleFMU
from fmuSimulation.configReader import configReader
import numpy as np
import sys
from torchModel import customTorchModel, FullyConnectedNetwork

In [6]:


config = os.path.abspath('Example.cfg')
cfg = configReader(config)
config = cfg.getAgent()
#if 0:   
config['framework'] = 'torch'
    #agent = ppo.PPO(env=ExampleFMU.gymFMU, config={"env_config": config, "num_workers": 1, "num_gpus": 0})
config['model'] = {}
config['model']['custom_model'] = FullyConnectedNetwork
config['model']['custom_model'] = customTorchModel
#config['model']['fcnet_hiddens'] = [10]
#
# config['model']['no_final_linear'] = False
#config['model']['custom_model'] = MyKerasModel
agent = pg.PG(env=ExampleFMU.gymFMU, config=config)

#agent = td3.TD3(env=ExampleFMU.gymFMU, config=config)
#agent = sac.SAC(env=ExampleFMU.gymFMU, config=config)


(1,)

In [47]:
x = MyAlgo()

TypeError: __init__() got an unexpected keyword argument 'observation_space'

In [40]:
from ray.rllib.policy import Policy
from ray.rllib.policy.torch_policy_template import build_torch_policy

class CustomPolicy(Policy):
    """Example of a custom policy written from scratch.

    You might find it more convenient to use the `build_tf_policy` and
    `build_torch_policy` helpers instead for a real policy, which are
    described in the next sections.
    """

    def __init__(self, observation_space, action_space, config):
        Policy.__init__(self, observation_space, action_space, config)
        # example parameter
        self.w = 1.0

    def compute_actions(self,
                        obs_batch,
                        state_batches,
                        prev_action_batch=None,
                        prev_reward_batch=None,
                        info_batch=None,
                        episodes=None,
                        **kwargs):
        # return action batch, RNN states, extra values to include in batch
        return [self.action_space.sample() for _ in obs_batch], [], {}

    def learn_on_batch(self, samples):
        # implement your learning code here
        return {}  # return stats

    def get_weights(self):
        return {"w": self.w}

    def set_weights(self, weights):
        self.w = weights["w"]



In [82]:
from ray.rllib.algorithms.algorithm import Algorithm
class MyAlgo(Algorithm):
    def get_default_policy_class(self, config):
        return PGTorchPolicy


class Testalg(pg.PG):
    @classmethod
    @override(Algorithm)
    def get_default_policy_class(cls, config):
        return PGTorchPolicy

from ray.rllib.algorithms.pg import pg
test = pg.with_updates(default_policy=Testalg)

AttributeError: module 'ray.rllib.algorithms.pg.pg' has no attribute 'with_updates'

In [81]:
from ray.rllib.algorithms.pg import pg
#config = {}
#config['model'] = {}
#config['model']['custom_model'] = PGTorchPolicy
Testalg(env='CartPole-v1', config=cfg)
#tune.Tuner(MyAlgo, param_space={"env": "CartPole-v1", "num_workers": 2}).fit()
num_outputs

AttributeError: type object 'PGConfig' has no attribute 'from_dict'

In [80]:
import logging
from typing import Dict, List, Type, Union, Optional, Tuple

from ray.rllib.evaluation.episode import Episode
from ray.rllib.utils.typing import AgentID
from ray.rllib.policy.torch_policy_v2 import TorchPolicyV2
from ray.rllib.utils.annotations import override
from ray.rllib.utils.numpy import convert_to_numpy
from ray.rllib.algorithms.pg.pg import PGConfig
from ray.rllib.algorithms.pg.utils import post_process_advantages
from ray.rllib.evaluation.postprocessing import Postprocessing
from ray.rllib.models.torch.torch_action_dist import TorchDistributionWrapper
from ray.rllib.models.modelv2 import ModelV2
from ray.rllib.policy import Policy
from ray.rllib.policy.sample_batch import SampleBatch
from ray.rllib.policy.torch_mixins import LearningRateSchedule
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.utils.typing import TensorType

class PGTorchPolicy(LearningRateSchedule, TorchPolicyV2):
    """PyTorch policy class used with PGTrainer."""

    def __init__(self, observation_space, action_space, config: PGConfig):

        # Enforce AlgorithmConfig for PG Policies.
        if isinstance(config, dict):
            config = PGConfig.from_dict(config)
            #pass
        TorchPolicyV2.__init__(
            self,
            observation_space,
            action_space,
            config,
            max_seq_len=config.model["max_seq_len"],
        )

        LearningRateSchedule.__init__(self, config.lr, config.lr_schedule)

        # TODO: Don't require users to call this manually.
        self._initialize_loss_from_dummy_batch()

    @override(TorchPolicyV2)
    def loss(
        self,
        model: ModelV2,
        dist_class: Type[TorchDistributionWrapper],
        train_batch: SampleBatch,
    ) -> Union[TensorType, List[TensorType]]:
        """The basic policy gradients loss function.
        Calculates the vanilla policy gradient loss based on:
        L = -E[ log(pi(a|s)) * A]
        Args:
            model: The Model to calculate the loss for.
            dist_class: The action distr. class.
            train_batch: The training data.
        Returns:
            Union[TensorType, List[TensorType]]: A single loss tensor or a list
                of loss tensors.
        """
        # Pass the training data through our model to get distribution parameters.
        dist_inputs, _ = model(train_batch)

        # Create an action distribution object.
        action_dist = dist_class(dist_inputs, model)

        # Calculate the vanilla PG loss based on:
        # L = -E[ log(pi(a|s)) * A]
        log_probs = action_dist.logp(train_batch[SampleBatch.ACTIONS])

        # Final policy loss.
        policy_loss = -torch.mean(log_probs * train_batch[Postprocessing.ADVANTAGES])

        # Store values for stats function in model (tower), such that for
        # multi-GPU, we do not override them during the parallel loss phase.
        model.tower_stats["policy_loss"] = policy_loss

        return policy_loss

    @override(TorchPolicyV2)
    def stats_fn(self, train_batch: SampleBatch) -> Dict[str, TensorType]:
        """Returns the calculated loss in a stats dict.
        Args:
            policy: The Policy object.
            train_batch: The data used for training.
        Returns:
            Dict[str, TensorType]: The stats dict.
        """

        return convert_to_numpy(
            {
                "policy_loss": torch.mean(
                    torch.stack(self.get_tower_stats("policy_loss"))
                ),
                "cur_lr": self.cur_lr,
            }
        )

    @override(TorchPolicyV2)
    def postprocess_trajectory(
        self,
        sample_batch: SampleBatch,
        other_agent_batches: Optional[
            Dict[AgentID, Tuple["Policy", SampleBatch]]
        ] = None,
        episode: Optional["Episode"] = None,
    ) -> SampleBatch:
        sample_batch = super().postprocess_trajectory(
            sample_batch, other_agent_batches, episode
        )
        return post_process_advantages(self, sample_batch, other_agent_batches, episode)